In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import savefig
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df = pd.read_csv("HRSS_anomalous_optimized_new.csv")
df_1 = df.drop(["Sl. No.", "Timestamp", "Training", "Labels"], axis = 1)
df_1.head(5)

In [ ]:
df_label = df["Labels"]

In [ ]:
#creating dataframe for boxplots
df_boxplot = pd.DataFrame(df_1[["O_w_HL_power", "O_w_BHR_power"]], columns = ["O_w_HL_power", "O_w_BHR_power"])
# detecting anomaly for a univariate variable
plt.figure(figsize = (12, 8))
df_boxplot.boxplot()

In [ ]:
# Using boxplot formula to label the outliers in the dataframe detected by boxplot for O_w_HL_power
Q1 = df_1["O_w_HL_power"].quantile(0.25)
Q3 = df_1["O_w_HL_power"].quantile(0.75)
IQR = Q3 - Q1

df_1["label_boxplot_O_w_HL_power"] = np.where((df_1["O_w_HL_power"] >= Q1 - 1.5 * IQR) & (df_1["O_w_HL_power"] <= Q3 + 1.5 *IQR), 0,1)
df_1.label_boxplot_O_w_HL_power.value_counts()

In [ ]:
# Using boxplot formula to label the outliers in the dataframe detected by boxplot for O_w_BHR_power
Q1 = df_1["O_w_BHR_power"].quantile(0.25)
Q3 = df_1["O_w_BHR_power"].quantile(0.75)
IQR = Q3 - Q1

df_1["label_boxplot_O_w_BHR_power"] = np.where((df_1["O_w_BHR_power"] >= Q1 - 1.5 * IQR) & (df_1["O_w_BHR_power"] <= Q3 + 1.5 *IQR), 0,1)
df_1.label_boxplot_O_w_BHR_power.value_counts()

In [ ]:
df_1.head(5)

In [ ]:
df_2 = df_1.drop(["label_boxplot_O_w_BHR_power", "label_boxplot_O_w_HL_power"], axis = 1)

### Using Isolation Forest for multivariate anomaly detection

In [ ]:
df_2.columns

In [ ]:
# scaling the data
scaler = MinMaxScaler()
df_2_scaled = scaler.fit_transform(df_2)
#transforming to dataframe
df_2_scaled = pd.DataFrame(df_2_scaled, columns = ['I_w_BLO_Weg', 'O_w_BLO_power', 'O_w_BLO_voltage', 'I_w_BHL_Weg',
       'O_w_BHL_power', 'O_w_BHL_voltage', 'I_w_BHR_Weg', 'O_w_BHR_power',
       'O_w_BHR_voltage', 'I_w_BRU_Weg', 'O_w_BRU_power', 'O_w_BRU_voltage',
       'I_w_HR_Weg', 'O_w_HR_power', 'O_w_HR_voltage', 'I_w_HL_Weg',
       'O_w_HL_power', 'O_w_HL_voltage'])

In [ ]:
df_2_scaled_w_label = df_2_scaled.join(df_label)

In [ ]:
# running isolation forest
clf=IsolationForest(n_estimators=50, max_samples=6000, contamination=float(.20), \
                        max_features=1.0, bootstrap=False, random_state=42, verbose=0, behaviour = "new")

In [ ]:
# taking all observations of the dataset to detect anomaly
clf.fit(df_2_scaled)
pred_all = clf.predict(df_2_scaled)
df_2_scaled_w_label["pred_if"] = pred_all

In [ ]:
df_2_scaled_w_label["pred_if"] = np.where(df_2_scaled_w_label["pred_if"] == 1, 0, 1)

In [ ]:
df_2_scaled_w_label.pred_if.value_counts()

In [ ]:
df_2_scaled_w_label["anomaly_score"] = clf.decision_function(df_2_scaled)

In [ ]:
df_2_scaled_w_label.head(5)

In [ ]:
# generating classification report and confusion matrix
print("Classification Report: \n", classification_report(df_2_scaled_w_label.Labels, df_2_scaled_w_label.pred_if))
print("Confusion Matrix: \n", confusion_matrix(df_2_scaled_w_label.Labels, df_2_scaled_w_label.pred_if))

In [ ]:
# importing the relevant datasets to excel
df_1.to_excel("df_boxplot_anomalies1.xlsx")
df_2_scaled_w_label.to_excel("df_isolation_forest_anomalies1.xlsx")